<a href="https://colab.research.google.com/github/AngelRosalesContreras/Practica_1-3/blob/main/Practica_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 1.3 (Transformada de Hough y Etiquetado de componentes conectados)**

In [3]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [4]:
def find(parent, i):
    if parent[i] == -1:
        return i
    return find(parent, parent[i])

def union(parent, x, y):
    x_set = find(parent, x)
    y_set = find(parent, y)
    if x_set != y_set:
        parent[x_set] = y_set

def label_connected_components(image_path):
    imagen = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # _, imagen_bin = cv2.threshold(imagen, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #
    # # Invertir la imagen binarizada
    # imagen_binarizada = 255 - imagen_bin

    # Aplicar la binarización de Otsu con umbralización
    _, imagen_binarizada1 = cv2.threshold(imagen, 20, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Encuentra y etiqueta las regiones conectadas
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(imagen_binarizada1)

    # Establece un umbral para el tamaño máximo de región
    tamano_maximo = 10000  # Puedes ajustar este valor según tus necesidades

    # Elimina las regiones que exceden el tamaño máximo
    for label in range(1, num_labels):  # Comenzamos desde 1 para omitir el fondo
        if stats[label, cv2.CC_STAT_AREA] > tamano_maximo:
            labels[labels == label] = 0  # Establece la región a 0 (negro)

    # Crea una nueva imagen binarizada con las regiones eliminadas
    imagen_binarizada = np.where(labels > 0, 255, 0).astype(np.uint8)

    height, width = imagen_binarizada.shape
    parent = [-1] * (height * width)
    component_colors = {}

    component_count = 0

    for y in range(height):
        for x in range(width):
            if imagen_binarizada[y, x] == 255:
                neighbors = []
                if x > 0:
                    neighbors.append((y, x - 1))
                if y > 0:
                    neighbors.append((y - 1, x))

                if not neighbors:
                    component_count += 1
                    component_colors[component_count] = np.random.randint(0, 256, 3)
                    imagen_binarizada[y, x] = component_count

                else:
                    neighbor_labels = []
                    for neighbor_y, neighbor_x in neighbors:
                        neighbor_label = imagen_binarizada[neighbor_y, neighbor_x]
                        if neighbor_label != 0:
                            neighbor_labels.append(neighbor_label)

                    if not neighbor_labels:
                        component_count += 1
                        component_colors[component_count] = np.random.randint(0, 256, 3)
                        imagen_binarizada[y, x] = component_count
                    else:
                        min_label = min(neighbor_labels)
                        imagen_binarizada[y, x] = min_label
                        for label in neighbor_labels:
                            if label != min_label:
                                union(parent, label, min_label)

    for i in range(1, component_count + 1):
        component_pixels = np.sum(imagen_binarizada == i)
        parent_label = find(parent, i)
        component_colors[i] = np.random.randint(0, 256, 3)  # Generar un nuevo color para cada componente
        print(f'Componente {i}: {component_pixels} píxeles, Padre: {parent_label}')

    # Crear una imagen en blanco del mismo tamaño
    colored_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Asignar colores a los componentes en la imagen original
    for i in range(1, component_count + 1):
        mask = (imagen_binarizada == i)
        colored_image[mask] = component_colors[i]

    return colored_image


image_path = 'placas/103.JPG'
labeled_image = label_connected_components(image_path)


# Mostrar la imagen binarizada invertida
cv2.imshow('Imagen Binarizada Invertida', labeled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Componente 1: 915 píxeles, Padre: 100
Componente 2: 443 píxeles, Padre: 100
Componente 3: 393 píxeles, Padre: 100
Componente 4: 49 píxeles, Padre: 100
Componente 5: 11 píxeles, Padre: 100
Componente 6: 938 píxeles, Padre: 100
Componente 7: 55 píxeles, Padre: 100
Componente 8: 13 píxeles, Padre: 100
Componente 9: 64 píxeles, Padre: 100
Componente 10: 68 píxeles, Padre: 100
Componente 11: 61 píxeles, Padre: 100
Componente 12: 33 píxeles, Padre: 100
Componente 13: 11 píxeles, Padre: 100
Componente 14: 69 píxeles, Padre: 100
Componente 15: 42 píxeles, Padre: 100
Componente 16: 43 píxeles, Padre: 100
Componente 17: 17 píxeles, Padre: 100
Componente 18: 22 píxeles, Padre: 100
Componente 19: 6 píxeles, Padre: 100
Componente 20: 456 píxeles, Padre: 100
Componente 21: 12 píxeles, Padre: 100
Componente 22: 4 píxeles, Padre: 100
Componente 23: 32 píxeles, Padre: 100
Componente 24: 264 píxeles, Padre: 100
Componente 25: 18 píxeles, Padre: 100
Componente 26: 29 píxeles, Padre: 100
Componente 27: 12

DisabledFunctionError: ignored